# Exercício - Mini Projeto de Análise de Dados

Vamos fazer um exercício completo de pandas para um miniprojeto de análise de dados.

Esse exercício vai obrigar a gente a usar boa parte dos conhecimento de pandas e até de outros módulos que já aprendemos ao longo do curso.

### O que temos?

Temos os dados de 2019 de uma empresa de prestação de serviços. 

- CadastroFuncionarios
- CadastroClientes
- BaseServiçosPrestados

Obs1: Para ler arquivos csv, temos o read_csv<br>
Obs2: Para ler arquivos xlsx (arquivos em excel normais, que não são padrão csv), temos o read_excel

### O que queremos saber/fazer?

1. Valor Total da Folha Salarial -> Qual foi o gasto total com salários de funcionários pela empresa? <br>
    Sugestão: calcule o salário total de cada funcionário, salário + benefícios + impostos, depois some todos os salários
    
    
2. Qual foi o faturamento da empresa?<br>
    Sugestão: calcule o faturamento total de cada serviço e depois some o faturamento de todos
    
    
3. Qual o % de funcionários que já fechou algum contrato?<br>
    Sugestão: na base de serviços temos o funcionário que fechou cada serviço. Mas nem todos os funcionários que a empresa tem já fecharam algum serviço.<br>
    . Na base de funcionários temos uma lista com todos os funcionários<br>
    . Queremos calcular Qtde_Funcionarios_Fecharam_Serviço / Qtde_Funcionários_Totais<br>
    . Para calcular a qtde de funcionários que fecharam algum serviço, use a base de serviços e conte quantos funcionários tem ali. Mas lembre-se, cada funcionário só pode ser contado uma única vez.<br><br>
    Dica: se você aplicar o método .unique() em uma variável que é apenas 1 coluna de um dataframe, ele vai excluir todos os valores duplicados daquela coluna.<br>
    Ex: unicos_colunaA = dataframe['colunaA'].unique() te dá como resposta uma lista com todos os itens da colunaA aparecendo uma única vez. Todos os valores repetidos da colunaA são excluidos da variável unicos_colunaA 
    
    
4. Calcule o total de contratos que cada área da empresa já fechou


5. Calcule o total de funcionários por área


6. Qual o ticket médio mensal (faturamento médio mensal) dos contratos?<br>
    Dica: .mean() calcula a média -> exemplo: media_colunaA = dataframe['colunaA'].mean()

Obs: Lembrando as opções mais usuais de encoding:<br>
encoding='latin1', encoding='ISO-8859-1', encoding='utf-8' ou então encoding='cp1252'

Observação Importante: Se o seu código der um erro na hora de importar os arquivos:<br>
- CadastroClientes.csv
- CadastroFuncionarios.csv

Use separador ";" (ponto e vírgula) para resolver

In [ ]:
import pandas as pd
import locale
from pathlib import path
import os

# Define a configuração regional para formatação de moeda
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

# Carrega os dados dos arquivos Excel e CSV
caminho_arquivo = path(os.getcwd()) / 'BaseServiçosPrestados.xlsx'
servicos_prestadosdf = pd.read_excel(caminho_arquivo)
clientesdf = pd.read_csv(r'C:\Users\lgcco\Downloads\Exercício pandas - Mini Projeto Análise de Dados/CadastroClientes.csv', sep=';')
funcionariosdf = pd.read_csv(r'C:\Users\lgcco\Downloads\Exercício pandas - Mini Projeto Análise de Dados/CadastroFuncionarios.csv', sep=';', decimal=',')

# Soma os valores das colunas 'Salario Base', 'Impostos', 'Beneficios', 'VT' e 'VR'
funcionariosdf['Salario Total'] = funcionariosdf[['Salario Base', 'Impostos', 'Beneficios', 'VT', 'VR']].sum(axis=1)

# Faz o merge das tabelas 'servicos_prestadosdf' e 'clientesdf' com base na coluna 'ID Cliente'
cliente_e_servicodf = pd.merge(servicos_prestadosdf, clientesdf, on='ID Cliente')

# Conta o número de IDs únicos de funcionários na tabela 'servicos_prestadosdf'
qtd_funcionarios_fechou_servico = servicos_prestadosdf['ID Funcionário'].nunique()

# Calcula o percentual de funcionários que fecharam contrato
percentual_funcionarios_fechou_servico = (qtd_funcionarios_fechou_servico / funcionariosdf['ID Funcionário'].count()) * 100

# Seleciona as colunas 'ID Funcionário' e 'Area' da tabela 'funcionariosdf'
coluna_selecionada = funcionariosdf[['ID Funcionário', 'Area']]

# Faz o merge das tabelas 'servicos_prestadosdf' e 'coluna_selecionada' com base na coluna 'ID Funcionário'
funcionario_e_servicodf = pd.merge(servicos_prestadosdf, coluna_selecionada, on='ID Funcionário')

# Conta o número de contratos por área e reseta o índice
qtd_contrato_por_area = funcionario_e_servicodf['Area'].value_counts().reset_index()
qtd_contrato_por_area.columns = ['Area', 'Quantidade']

# Conta o número de funcionários por área e reseta o índice
qtd_funcionario_por_area = funcionariosdf['Area'].value_counts().reset_index()
qtd_funcionario_por_area.columns = ['Area', 'Quantidade']

# Calcula a média da coluna 'Valor Contrato Mensal' na tabela 'clientesdf'
faturamento_medio_mensal = clientesdf['Valor Contrato Mensal'].mean()

# Imprime os resultados formatados
print('O gasto total com salários é de {}'.format(locale.currency(sum(funcionariosdf['Salario Total']), grouping=True)))
print('-' * 70)
print('O faturamento total é de {}'.format(locale.currency(sum(cliente_e_servicodf['Tempo Total de Contrato (Meses)'] * cliente_e_servicodf['Valor Contrato Mensal']), grouping=True)))
print('-' * 70)
print('O percentual de funcionários que já fechou algum contrato é de {:.2f}%'.format(percentual_funcionarios_fechou_servico))
print('-' * 70)

# Itera sobre os resultados de contratos por área e imprime
print('A quantidade de contratos fechados por área é:')
for _, row in qtd_contrato_por_area.iterrows():
    print(f'A Área {row["Area"]} fechou {row["Quantidade"]} contratos')
print('-' * 70)

# Itera sobre os resultados de funcionários por área e imprime
print('A quantidade de funcionários por área é:')
for _, row in qtd_funcionario_por_area.iterrows():
    print(f'A Área {row["Area"]} tem {row["Quantidade"]} funcionários')
print('-' * 70)

# Imprime o ticket médio mensal
print('O ticket médio mensal é de {}'.format(locale.currency(faturamento_medio_mensal, grouping=True)))
